<div align='left' ><font size='70'>Setup and function</font></div>

In [ ]:
PKL_FILE = ['', 'full_batch1_big_label_list.pkl']
CSV_FILE = 'batch2_onlyvaild.csv'
active_learning_target_label = 'FullNegative'
BEST_MODEL_DIR = 'clfmodel/2/best_model'
ALL_MODEL_DIR = ['clfmodel/2/best_model', 'clfmodel/0/best_model', 'clfmodel/4/best_model']
TEXT_HEADER = 'raw_text'

import numpy as np
import pandas as pd  # type: ignore

print('import all DONE!')
import ast
import copy
import os
import pickle

from scipy.stats import entropy  # type: ignore
from simpletransformers.classification import (  # type: ignore
    ClassificationArgs, MultiLabelClassificationModel)

os.chdir(r"/data/V10")

pkl_file = open(PKL_FILE[1], 'rb')
big_label_list = pickle.load(pkl_file)
pkl_file.close()
print(big_label_list)


#Functions
def add_string_preditcion(inputDF: pd.DataFrame) -> pd.DataFrame:
    predict_string = []
    for i in inputDF.index:
        predict_onehot = ast.literal_eval(inputDF.at[i, 'predict'])
        builder_string = ""
        for j in range(len(predict_onehot)):
            if predict_onehot[j] == 1:
                if builder_string == '':
                    builder_string = big_label_list[j]
                else:
                    builder_string = builder_string + ', ' + big_label_list[j]
        if builder_string == '':
            builder_string = 'FullNegative'
        #if sum(predict_onehot)>=2:
        predict_string.append(builder_string)
    inputDF['predict_string'] = predict_string
    return inputDF


def Uncertainty_Sampling(inputDF: pd.DataFrame) -> pd.DataFrame:
    copyDF = copy.deepcopy(inputDF)
    copyDF.reset_index(inplace=True, drop=True)
    classification_uncertainty = []
    classification_margin = []
    classification_entropy = []
    model_args = {"silent": False, }
    predict_string_list = inputDF[TEXT_HEADER].values.tolist()
    model = MultiLabelClassificationModel("roberta", BEST_MODEL_DIR, args=model_args)
    predictions, raw_outputs = model.predict(predict_string_list)
    for s in raw_outputs:
        classification_uncertainty.append(1 - max(s))
        classification_entropy.append(entropy(s))
        s.sort()
        classification_margin.append(1 - (s[-1] - s[-2]))
    predict_string = []
    for i in copyDF.index:
        predict_onehot = predictions[i]
        builder_string = ""
        for j in range(len(predict_onehot)):
            if predict_onehot[j] == 1:
                if builder_string == '':
                    builder_string = big_label_list[j]
                else:
                    builder_string = builder_string + ', ' + big_label_list[j]
        if builder_string == '':
            builder_string = 'FullNegative'
        #if sum(predict_onehot)>=2:
        predict_string.append(builder_string)
    raw_list = raw_outputs.tolist()
    copyDF['predict_string'] = predict_string
    copyDF['raw_outputs'] = raw_list
    copyDF['classification_uncertainty'] = classification_uncertainty
    copyDF['classification_margin'] = classification_margin
    copyDF['classification_entropy'] = classification_entropy
    return copyDF


<div align='left' ><font size='70'>Active learning </font></div>

In [ ]:
data = pd.read_csv(CSV_FILE)
data.reset_index(inplace=True, drop=True)
if 'predict' not in list(data.columns):
    predict_fill_null = ['Null_predict'] * data.shape[0]
    data['predict'] = predict_fill_null
else:
    data_with_predicted_label = add_string_preditcion(data)
    data_with_target_predicted_label = data_with_predicted_label[data_with_predicted_label['predict_string'].str.contains(active_learning_target_label)]
    data_with_target_predicted_label.reset_index(inplace=True, drop=True)
    data = data_with_target_predicted_label
data_with_uncertanty = Uncertainty_Sampling(data)
data_with_uncertanty = data_with_uncertanty
number_col = ['classification_uncertainty', 'classification_margin','classification_entropy']

while True in data_with_uncertanty[TEXT_HEADER].str.contains("\n").tolist():
    data_with_uncertanty = data_with_uncertanty.replace('\n', '', regex=True)

sum_of_total = []
product_of_total = []

for i in data_with_uncertanty.index:
    thesum = 0
    for s in number_col:
        thesum = thesum + data_with_uncertanty.at[i, s]
    sum_of_total.append(thesum)
    product = 1
    for s in number_col:
        product = product * data_with_uncertanty.at[i, s]
    product_of_total.append(product)
data_with_uncertanty['Sum_of_uncertainty_value'] = sum_of_total
data_with_uncertanty['Product_of_uncertainty_value'] = product_of_total

In [ ]:
#Save the result
active_learning_target_label = active_learning_target_label.replace(" ", "")
finalstr = CSV_FILE.split('.csv')[0] + '_' + active_learning_target_label + '_active_learning.csv'
data_with_uncertanty.to_csv(finalstr, index=False, encoding='utf-8-sig')